# 0 - QB - Crossvals

## Importation des modules

In [ ]:
# Importation des modules
# Modules de base
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

# Configuration de l'affichage
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)  # Pour la reproductibilité

# Importation des classes de validation croisée
import sys
sys.path.append('../')
from tsforecast.crossvals import (
    TSOutOfSampleSplit, TSInSampleSplit,
    PanelOutOfSampleSplit, PanelInSampleSplit,
    PanelOutOfSampleSplitPerEntity, PanelInSampleSplitPerEntity
)

## Génération de données synthétiques

Ce notebook illustre l'utilisation des classes de validation croisée pour les séries temporelles et les données de panel. Nous commençons par générer des données synthétiques avec différentes caractéristiques pour démontrer le comportement des différentes méthodes.

In [ ]:
### 1. Génération de séries temporelles synthétiques

def generate_time_series(start_date='2020-01-01', periods=252, freq='D', trend_strength=0.02, 
                        seasonal_period=30, seasonal_strength=0.5, noise_std=1.0, 
                        ar_coefficient=0.7, add_outliers=False, outlier_prob=0.05):
    """
    Generate synthetic time series with various patterns.
    
    Args:
        start_date: Start date for the time series
        periods: Number of time periods
        freq: Frequency ('D' for daily, 'M' for monthly, etc.)
        trend_strength: Strength of linear trend component
        seasonal_period: Period of seasonal pattern
        seasonal_strength: Strength of seasonal component
        noise_std: Standard deviation of random noise
        ar_coefficient: Autoregressive coefficient for AR(1) process
        add_outliers: Whether to add random outliers
        outlier_prob: Probability of outliers
    
    Returns:
        pd.Series: Time series with DatetimeIndex
    """
    # Création de l'index temporel
    dates = pd.date_range(start=start_date, periods=periods, freq=freq)
    
    # Composante tendance linéaire
    trend = trend_strength * np.arange(periods)
    
    # Composante saisonnière
    seasonal = seasonal_strength * np.sin(2 * np.pi * np.arange(periods) / seasonal_period)
    
    # Processus autorégressif AR(1) pour la persistance
    ar_process = np.zeros(periods)
    ar_process[0] = np.random.normal(0, noise_std)
    for i in range(1, periods):
        ar_process[i] = ar_coefficient * ar_process[i-1] + np.random.normal(0, noise_std)
    
    # Combinaison des composantes
    values = trend + seasonal + ar_process
    
    # Ajout d'outliers aléatoires
    if add_outliers:
        outlier_mask = np.random.random(periods) < outlier_prob
        outlier_values = np.random.normal(0, 5 * noise_std, size=np.sum(outlier_mask))
        values[outlier_mask] += outlier_values
    
    return pd.Series(values, index=dates, name='value')

# Génération de différents types de séries temporelles
print("📊 Génération de séries temporelles avec différentes caractéristiques...")

# Série 1: Trend fort, faible saisonnalité
ts_trend = generate_time_series(
    start_date='2020-01-01', periods=200, freq='D',
    trend_strength=0.05, seasonal_strength=0.2, noise_std=0.5,
    ar_coefficient=0.8
)

# Série 2: Saisonnalité forte, trend faible
ts_seasonal = generate_time_series(
    start_date='2020-01-01', periods=200, freq='D',
    trend_strength=0.01, seasonal_strength=1.5, seasonal_period=50,
    noise_std=0.3, ar_coefficient=0.6
)

# Série 3: Série très bruitée avec outliers
ts_noisy = generate_time_series(
    start_date='2020-01-01', periods=200, freq='D',
    trend_strength=0.02, seasonal_strength=0.5, noise_std=2.0,
    ar_coefficient=0.3, add_outliers=True, outlier_prob=0.08
)

# Série 4: Série stationnaire (pas de trend)
ts_stationary = generate_time_series(
    start_date='2020-01-01', periods=200, freq='D',
    trend_strength=0.0, seasonal_strength=0.8, noise_std=1.0,
    ar_coefficient=0.5
)

print(f"✅ Génération terminée:")
print(f"  - Série avec trend: {len(ts_trend)} observations de {ts_trend.index[0].date()} à {ts_trend.index[-1].date()}")
print(f"  - Série saisonnière: {len(ts_seasonal)} observations")
print(f"  - Série bruitée: {len(ts_noisy)} observations")
print(f"  - Série stationnaire: {len(ts_stationary)} observations")

In [ ]:
# Visualisation des séries temporelles générées
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Séries temporelles synthétiques avec différentes caractéristiques', fontsize=16)

# Série avec trend
axes[0, 0].plot(ts_trend.index, ts_trend.values, linewidth=1.5, color='blue')
axes[0, 0].set_title('Série avec trend fort')
axes[0, 0].set_ylabel('Valeur')
axes[0, 0].grid(True, alpha=0.3)

# Série saisonnière
axes[0, 1].plot(ts_seasonal.index, ts_seasonal.values, linewidth=1.5, color='green')
axes[0, 1].set_title('Série avec saisonnalité forte')
axes[0, 1].set_ylabel('Valeur')
axes[0, 1].grid(True, alpha=0.3)

# Série bruitée
axes[1, 0].plot(ts_noisy.index, ts_noisy.values, linewidth=1.5, color='red')
axes[1, 0].set_title('Série bruitée avec outliers')
axes[1, 0].set_ylabel('Valeur')
axes[1, 0].set_xlabel('Date')
axes[1, 0].grid(True, alpha=0.3)

# Série stationnaire
axes[1, 1].plot(ts_stationary.index, ts_stationary.values, linewidth=1.5, color='purple')
axes[1, 1].set_title('Série stationnaire')
axes[1, 1].set_ylabel('Valeur')
axes[1, 1].set_xlabel('Date')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2. Génération de données de panel synthétiques

Les données de panel combinent plusieurs entités observées sur plusieurs périodes temporelles. Nous allons créer des datasets de panel avec différentes caractéristiques pour illustrer le comportement des classes de validation croisée.

In [ ]:
def generate_panel_data(entities=['A', 'B', 'C'], start_date='2020-01-01', periods=100, 
                        freq='D', heterogeneous_effects=True, common_trend=True, 
                        entity_specific_seasonality=True, cross_sectional_correlation=0.3,
                        missing_data_prob=0.0):
    """
    Generate synthetic panel data with various characteristics.
    
    Args:
        entities: List of entity identifiers
        start_date: Start date for the panel
        periods: Number of time periods per entity
        freq: Frequency of observations
        heterogeneous_effects: Whether entities have different baseline levels
        common_trend: Whether to include a common trend across entities
        entity_specific_seasonality: Whether seasonality patterns differ by entity
        cross_sectional_correlation: Correlation between entity shocks
        missing_data_prob: Probability of missing observations
    
    Returns:
        pd.DataFrame: Panel data with MultiIndex (entity, date)
    """
    # Création de l'index temporel
    dates = pd.date_range(start=start_date, periods=periods, freq=freq)
    
    # Création du MultiIndex (entity, date)
    index = pd.MultiIndex.from_product([entities, dates], names=['entity', 'date'])
    
    # Initialisation du DataFrame
    n_total = len(entities) * periods
    panel_data = pd.DataFrame(index=index)
    
    # Génération des effets fixes par entité (hétérogénéité)
    if heterogeneous_effects:
        entity_effects = {entity: np.random.normal(0, 2) for entity in entities}
    else:
        entity_effects = {entity: 0 for entity in entities}
    
    # Tendance commune
    if common_trend:
        common_trend_values = 0.02 * np.arange(periods)
    else:
        common_trend_values = np.zeros(periods)
    
    # Génération de chocs corrélés entre entités
    if cross_sectional_correlation > 0:
        # Chocs communs
        common_shocks = np.random.normal(0, 1, periods)
        # Chocs idiosyncratiques
        idiosyncratic_shocks = {
            entity: np.random.normal(0, 1, periods) 
            for entity in entities
        }
    
    # Construction des séries pour chaque entité
    values = []
    entity_labels = []
    date_labels = []
    
    for entity in entities:
        # Effet fixe de l'entité
        entity_effect = entity_effects[entity]
        
        # Saisonnalité spécifique à l'entité
        if entity_specific_seasonality:
            # Période et amplitude différentes selon l'entité
            seasonal_period = 20 + hash(entity) % 40  # Entre 20 et 60
            seasonal_amplitude = 0.5 + (hash(entity) % 100) / 200  # Entre 0.5 et 1.0
        else:
            seasonal_period = 30
            seasonal_amplitude = 0.5
        
        seasonal_values = seasonal_amplitude * np.sin(2 * np.pi * np.arange(periods) / seasonal_period)
        
        # Processus autorégressif spécifique à l'entité
        ar_coef = 0.5 + (hash(entity) % 50) / 100  # Entre 0.5 et 1.0
        ar_process = np.zeros(periods)
        ar_process[0] = np.random.normal(0, 0.5)
        for t in range(1, periods):
            ar_process[t] = ar_coef * ar_process[t-1] + np.random.normal(0, 0.5)
        
        # Combinaison des composantes
        if cross_sectional_correlation > 0:
            # Chocs avec corrélation croisée
            correlated_shocks = (
                np.sqrt(cross_sectional_correlation) * common_shocks +
                np.sqrt(1 - cross_sectional_correlation) * idiosyncratic_shocks[entity]
            )
        else:
            correlated_shocks = np.random.normal(0, 1, periods)
        
        entity_values = (
            entity_effect + 
            common_trend_values + 
            seasonal_values + 
            ar_process + 
            correlated_shocks
        )
        
        # Ajout de données manquantes
        if missing_data_prob > 0:
            missing_mask = np.random.random(periods) < missing_data_prob
            entity_values[missing_mask] = np.nan
        
        values.extend(entity_values)
        entity_labels.extend([entity] * periods)
        date_labels.extend(dates)
    
    # Création du DataFrame final
    panel_data['value'] = values
    panel_data['entity'] = entity_labels
    panel_data['date'] = date_labels
    
    # Ajout de variables explicatives
    panel_data['lag_value'] = panel_data.groupby('entity')['value'].shift(1)
    panel_data['trend'] = np.tile(np.arange(periods), len(entities))
    panel_data['month'] = panel_data['date'].dt.month
    
    return panel_data[['value', 'lag_value', 'trend', 'month']]

# Génération de différents types de données de panel
print("📊 Génération de données de panel avec différentes caractéristiques...")

# Panel 1: Données équilibrées avec effets hétérogènes
entities_small = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']
panel_balanced = generate_panel_data(
    entities=entities_small,
    start_date='2020-01-01',
    periods=120,
    freq='D',
    heterogeneous_effects=True,
    common_trend=True,
    entity_specific_seasonality=True,
    cross_sectional_correlation=0.4
)

# Panel 2: Grand panel avec nombreuses entités
entities_large = [f'Entity_{i:02d}' for i in range(1, 21)]  # 20 entités
panel_large = generate_panel_data(
    entities=entities_large,
    start_date='2020-01-01',
    periods=100,
    freq='D',
    heterogeneous_effects=True,
    common_trend=True,
    entity_specific_seasonality=False,  # Saisonnalité commune
    cross_sectional_correlation=0.6
)

# Panel 3: Données avec observations manquantes
panel_missing = generate_panel_data(
    entities=['Entity_A', 'Entity_B', 'Entity_C'],
    start_date='2020-01-01',
    periods=80,
    freq='D',
    heterogeneous_effects=True,
    common_trend=False,
    entity_specific_seasonality=True,
    cross_sectional_correlation=0.2,
    missing_data_prob=0.05  # 5% de données manquantes
)

print(f"✅ Génération de panels terminée:")
print(f"  - Panel équilibré: {panel_balanced.shape[0]} observations, {len(entities_small)} entités")
print(f"  - Grand panel: {panel_large.shape[0]} observations, {len(entities_large)} entités")
print(f"  - Panel avec données manquantes: {panel_missing.shape[0]} observations, {panel_missing['value'].notna().sum()} valides")

# Affichage des premières observations de chaque panel
print(f"\n📋 Aperçu des données:")
print(f"\nPanel équilibré (premières 10 observations):")
print(panel_balanced.head(10))
print(f"\nPanel avec données manquantes (aperçu):")
print(panel_missing[panel_missing['value'].isna()].head())

In [ ]:
# Visualisation des données de panel
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Données de panel synthétiques', fontsize=16)

# Panel équilibré - quelques entités
entities_to_plot = entities_small[:3]
for i, entity in enumerate(entities_to_plot):
    entity_data = panel_balanced.xs(entity, level='entity')
    axes[0, 0].plot(entity_data.index, entity_data['value'], 
                   label=entity, linewidth=1.5, alpha=0.8)
axes[0, 0].set_title('Panel équilibré (échantillon d\'entités)')
axes[0, 0].set_ylabel('Valeur')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Distribution des valeurs par entité (panel équilibré)
panel_balanced.reset_index().boxplot(column='value', by='entity', ax=axes[0, 1])
axes[0, 1].set_title('Distribution par entité (panel équilibré)')
axes[0, 1].set_ylabel('Valeur')
axes[0, 1].set_xlabel('Entité')

# Grand panel - moyennes par période
large_panel_means = panel_large.groupby('date')['value'].agg(['mean', 'std']).reset_index()
axes[1, 0].plot(large_panel_means['date'], large_panel_means['mean'], 
               color='blue', linewidth=1.5, label='Moyenne')
axes[1, 0].fill_between(large_panel_means['date'], 
                       large_panel_means['mean'] - large_panel_means['std'],
                       large_panel_means['mean'] + large_panel_means['std'],
                       alpha=0.3, color='blue', label='±1 écart-type')
axes[1, 0].set_title('Grand panel (20 entités) - Statistiques agrégées')
axes[1, 0].set_ylabel('Valeur')
axes[1, 0].set_xlabel('Date')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Panel avec données manquantes
missing_stats = panel_missing.groupby('entity')['value'].apply(
    lambda x: x.notna().sum() / len(x) * 100
).reset_index()
missing_stats.columns = ['entity', 'completeness_pct']
bars = axes[1, 1].bar(missing_stats['entity'], missing_stats['completeness_pct'])
axes[1, 1].set_title('Complétude des données par entité (%)')
axes[1, 1].set_ylabel('Pourcentage de données valides')
axes[1, 1].set_xlabel('Entité')
axes[1, 1].set_ylim(0, 100)
# Ajout des valeurs sur les barres
for bar, value in zip(bars, missing_stats['completeness_pct']):
    axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                   f'{value:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Démonstration des classes de validation croisée pour séries temporelles

Les classes `TSOutOfSampleSplit` et `TSInSampleSplit` sont spécialement conçues pour les séries temporelles. Elles respectent l'ordre temporel et permettent différentes configurations selon les besoins d'évaluation.

### 3.1 TSOutOfSampleSplit - Validation hors échantillon

La validation **out-of-sample** (hors échantillon) est la méthode standard pour évaluer les modèles de prévision. L'entraînement se fait **strictement sur le passé** et le test sur le **futur**, respectant ainsi l'ordre temporel naturel.

In [ ]:
def analyze_split_characteristics(X, train_indices, test_indices, split_name):
    """Fonction utilitaire pour analyser les caractéristiques d'une séparation."""
    train_dates = X.index[train_indices]
    test_dates = X.index[test_indices]
    
    print(f"\n📊 {split_name}:")
    print(f"  - Taille d'entraînement: {len(train_indices)} observations")
    print(f"  - Taille de test: {len(test_indices)} observations")
    print(f"  - Période d'entraînement: {train_dates[0].date()} à {train_dates[-1].date()}")
    print(f"  - Période de test: {test_dates[0].date()} à {test_dates[-1].date()}")
    
    # Vérification de l'ordre temporel
    gap_days = (test_dates[0] - train_dates[-1]).days
    print(f"  - Gap entre train et test: {gap_days} jours")
    
    return {
        'train_size': len(train_indices),
        'test_size': len(test_indices),
        'train_start': train_dates[0],
        'train_end': train_dates[-1],
        'test_start': test_dates[0],
        'test_end': test_dates[-1],
        'gap_days': gap_days
    }

print("🔍 DÉMONSTRATION: TSOutOfSampleSplit avec différents paramètres")
print("="*70)

# Utilisation de la série avec trend pour les démonstrations
X = ts_trend.to_frame('value')
print(f"Série utilisée: {len(X)} observations de {X.index[0].date()} à {X.index[-1].date()}")

# Configuration 1: Split basique avec n_splits
print(f"\n{'='*50}")
print("CONFIGURATION 1: Split basique avec n_splits")
print(f"{'='*50}")

splitter1 = TSOutOfSampleSplit(n_splits=3, test_size=20)
splits_info = []

for i, (train_idx, test_idx) in enumerate(splitter1.split(X)):
    split_info = analyze_split_characteristics(X, train_idx, test_idx, f"Split {i+1}")
    splits_info.append(split_info)

# Configuration 2: Avec gap pour éviter le data leakage
print(f"\n{'='*50}")
print("CONFIGURATION 2: Avec gap pour éviter le data leakage")
print(f"{'='*50}")

splitter2 = TSOutOfSampleSplit(n_splits=3, test_size=15, gap=5)
print("⚠️  Gap = 5 jours entre l'entraînement et le test")

for i, (train_idx, test_idx) in enumerate(splitter2.split(X)):
    analyze_split_characteristics(X, train_idx, test_idx, f"Split avec gap {i+1}")

# Configuration 3: Fenêtre d'entraînement limitée (rolling window)
print(f"\n{'='*50}")
print("CONFIGURATION 3: Fenêtre d'entraînement limitée (rolling window)")
print(f"{'='*50}")

splitter3 = TSOutOfSampleSplit(n_splits=3, test_size=15, max_train_size=50, gap=2)
print("📏 max_train_size = 50 observations (fenêtre glissante)")

for i, (train_idx, test_idx) in enumerate(splitter3.split(X)):
    analyze_split_characteristics(X, train_idx, test_idx, f"Rolling window {i+1}")

# Configuration 4: Test sur des dates spécifiques
print(f"\n{'='*50}")
print("CONFIGURATION 4: Test sur des dates spécifiques")
print(f"{'='*50}")

specific_test_dates = ['2020-03-01', '2020-04-15', '2020-06-01']
splitter4 = TSOutOfSampleSplit(test_indices=specific_test_dates, test_size=10, gap=3)
print(f"🎯 Dates de test spécifiques: {specific_test_dates}")

for i, (train_idx, test_idx) in enumerate(splitter4.split(X)):
    analyze_split_characteristics(X, train_idx, test_idx, f"Test spécifique {i+1}")

In [ ]:
# Visualisation des différentes configurations de TSOutOfSampleSplit
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Visualisation des différentes configurations TSOutOfSampleSplit', fontsize=16)

configurations = [
    (splitter1, "Split basique (n_splits=3)", axes[0, 0]),
    (splitter2, "Avec gap=5", axes[0, 1]), 
    (splitter3, "Fenêtre limitée (max_train_size=50)", axes[1, 0]),
    (splitter4, "Dates spécifiques", axes[1, 1])
]

colors = ['blue', 'red', 'green', 'orange', 'purple']

for splitter, title, ax in configurations:
    # Plot de la série complète
    ax.plot(X.index, X['value'], color='lightgray', alpha=0.5, linewidth=1, label='Données complètes')
    
    # Plot des splits
    for i, (train_idx, test_idx) in enumerate(splitter.split(X)):
        train_data = X.iloc[train_idx]
        test_data = X.iloc[test_idx]
        
        # Données d'entraînement
        ax.plot(train_data.index, train_data['value'], 
               color=colors[i], alpha=0.7, linewidth=2, 
               label=f'Train {i+1}' if i < 3 else None)
        
        # Données de test
        ax.scatter(test_data.index, test_data['value'], 
                  color=colors[i], s=30, alpha=0.9, marker='o',
                  edgecolors='black', linewidth=0.5,
                  label=f'Test {i+1}' if i < 3 else None)
    
    ax.set_title(title)
    ax.set_ylabel('Valeur')
    ax.grid(True, alpha=0.3)
    if ax in [axes[1, 0], axes[1, 1]]:
        ax.set_xlabel('Date')
    
    # Légende seulement pour le premier graphique
    if ax == axes[0, 0]:
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

# Résumé des caractéristiques importantes
print("\n" + "="*70)
print("📋 RÉSUMÉ DES CARACTÉRISTIQUES IMPORTANTES")
print("="*70)
print("\n✅ Points clés à retenir:")
print("  1. Out-of-sample: l'entraînement précède TOUJOURS le test temporellement")
print("  2. Gap: permet d'éviter le data leakage en laissant un intervalle")
print("  3. max_train_size: limite la fenêtre d'entraînement (rolling window)")
print("  4. test_indices: permet de tester sur des périodes spécifiques")
print("  5. Les splits respectent l'ordre chronologique des données")

### 3.2 TSInSampleSplit - Validation dans l'échantillon

La validation **in-sample** (dans l'échantillon) inclut la période de test dans les données d'entraînement. Cette approche est utile pour l'évaluation historique et la calibration de modèles, où l'information future est disponible.

In [ ]:
print("🔍 DÉMONSTRATION: TSInSampleSplit - Validation dans l'échantillon")
print("="*70)

def analyze_insample_split(X, train_indices, test_indices, split_name):
    """Fonction pour analyser les splits in-sample."""
    train_dates = X.index[train_indices]
    test_dates = X.index[test_indices]
    
    print(f"\n📊 {split_name}:")
    print(f"  - Taille d'entraînement: {len(train_indices)} observations")
    print(f"  - Taille de test: {len(test_indices)} observations")
    print(f"  - Période d'entraînement: {train_dates[0].date()} à {train_dates[-1].date()}")
    print(f"  - Période de test: {test_dates[0].date()} à {test_dates[-1].date()}")
    
    # Vérification que le test est inclus dans l'entraînement
    test_in_train = all(idx in train_indices for idx in test_indices)
    print(f"  - Test inclus dans train: {'✅ Oui' if test_in_train else '❌ Non'}")
    
    return test_in_train

# Configuration 1: In-sample basique
print(f"\n{'='*50}")
print("CONFIGURATION 1: In-sample basique")
print(f"{'='*50}")

insample_splitter1 = TSInSampleSplit(test_size=20)
print("📖 Les données de test sont incluses dans l'entraînement")

for i, (train_idx, test_idx) in enumerate(insample_splitter1.split(X)):
    analyze_insample_split(X, train_idx, test_idx, f"In-sample split {i+1}")

# Configuration 2: In-sample avec fenêtre d'entraînement limitée
print(f"\n{'='*50}")
print("CONFIGURATION 2: In-sample avec max_train_size")
print(f"{'='*50}")

insample_splitter2 = TSInSampleSplit(test_size=15, max_train_size=80)
print("📏 Entraînement limité mais inclut toujours la période de test")

for i, (train_idx, test_idx) in enumerate(insample_splitter2.split(X)):
    analyze_insample_split(X, train_idx, test_idx, f"Limited in-sample {i+1}")

# Configuration 3: In-sample sur dates spécifiques
print(f"\n{'='*50}")
print("CONFIGURATION 3: In-sample sur dates spécifiques")
print(f"{'='*50}")

specific_dates = ['2020-04-01']
insample_splitter3 = TSInSampleSplit(test_indices=specific_dates, test_size=14)
print(f"🎯 Test sur période spécifique: {specific_dates[0]} (14 jours)")

for i, (train_idx, test_idx) in enumerate(insample_splitter3.split(X)):
    analyze_insample_split(X, train_idx, test_idx, f"Specific in-sample {i+1}")

# Configuration 4: Comparaison de plusieurs dates spécifiques
print(f"\n{'='*50}")
print("CONFIGURATION 4: Multiples dates spécifiques")
print(f"{'='*50}")

multiple_dates = ['2020-02-15', '2020-03-15', '2020-05-01']
insample_splitter4 = TSInSampleSplit(test_indices=multiple_dates, test_size=7)
print(f"🎯 Tests sur: {multiple_dates}")

for i, (train_idx, test_idx) in enumerate(insample_splitter4.split(X)):
    analyze_insample_split(X, train_idx, test_idx, f"Multiple dates {i+1}")

In [ ]:
# Visualisation comparative: Out-of-sample vs In-sample
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Comparaison Out-of-sample vs In-sample', fontsize=16)

# Données de test communes pour la comparaison
test_date_comparison = '2020-04-01'
test_size_comparison = 10

# Out-of-sample
out_splitter = TSOutOfSampleSplit(test_indices=[test_date_comparison], test_size=test_size_comparison, gap=2)
in_splitter = TSInSampleSplit(test_indices=[test_date_comparison], test_size=test_size_comparison)

# Visualisation Out-of-sample
for train_idx, test_idx in out_splitter.split(X):
    train_data = X.iloc[train_idx]
    test_data = X.iloc[test_idx]
    
    axes[0, 0].plot(X.index, X['value'], color='lightgray', alpha=0.5, linewidth=1, label='Données complètes')
    axes[0, 0].plot(train_data.index, train_data['value'], color='blue', alpha=0.8, linewidth=2, label='Train')
    axes[0, 0].scatter(test_data.index, test_data['value'], color='red', s=40, alpha=0.9, 
                      edgecolors='black', linewidth=0.5, label='Test')
    
    # Mise en évidence du gap
    if len(train_data) > 0 and len(test_data) > 0:
        gap_start = train_data.index[-1]
        gap_end = test_data.index[0]
        axes[0, 0].axvspan(gap_start, gap_end, alpha=0.3, color='yellow', label='Gap')

axes[0, 0].set_title('Out-of-sample: Train → Gap → Test')
axes[0, 0].set_ylabel('Valeur')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Visualisation In-sample
for train_idx, test_idx in in_splitter.split(X):
    train_data = X.iloc[train_idx]
    test_data = X.iloc[test_idx]
    
    axes[0, 1].plot(X.index, X['value'], color='lightgray', alpha=0.5, linewidth=1, label='Données complètes')
    axes[0, 1].plot(train_data.index, train_data['value'], color='blue', alpha=0.8, linewidth=2, label='Train')
    axes[0, 1].scatter(test_data.index, test_data['value'], color='red', s=40, alpha=0.9, 
                      edgecolors='black', linewidth=0.5, label='Test')
    
    # Mise en évidence de l'overlap
    overlap_indices = np.intersect1d(train_idx, test_idx)
    if len(overlap_indices) > 0:
        overlap_data = X.iloc[overlap_indices]
        axes[0, 1].scatter(overlap_data.index, overlap_data['value'], color='purple', s=60, 
                          alpha=0.7, marker='s', edgecolors='black', linewidth=1, label='Overlap (Test dans Train)')

axes[0, 1].set_title('In-sample: Test inclus dans Train')
axes[0, 1].set_ylabel('Valeur')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Graphique de distribution des erreurs simulées
print("\n🧮 Simulation d'évaluation avec différents modèles...")

# Simulation d'erreurs pour démontrer l'impact
np.random.seed(42)
n_simulations = 1000

# Erreurs out-of-sample (plus réalistes)
out_sample_errors = np.random.normal(0, 1.5, n_simulations)  # Plus d'incertitude

# Erreurs in-sample (généralement plus faibles)
in_sample_errors = np.random.normal(0, 0.8, n_simulations)   # Moins d'incertitude

axes[1, 0].hist(out_sample_errors, bins=50, alpha=0.7, color='red', label='Out-of-sample', density=True)
axes[1, 0].hist(in_sample_errors, bins=50, alpha=0.7, color='blue', label='In-sample', density=True)
axes[1, 0].set_title('Distribution des erreurs de prédiction')
axes[1, 0].set_xlabel('Erreur')
axes[1, 0].set_ylabel('Densité')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Statistiques comparatives
out_mae = np.mean(np.abs(out_sample_errors))
in_mae = np.mean(np.abs(in_sample_errors))
out_mse = np.mean(out_sample_errors**2)
in_mse = np.mean(in_sample_errors**2)

metrics = ['MAE', 'MSE']
out_values = [out_mae, out_mse]
in_values = [in_mae, in_mse]

x = np.arange(len(metrics))
width = 0.35

bars1 = axes[1, 1].bar(x - width/2, out_values, width, label='Out-of-sample', color='red', alpha=0.7)
bars2 = axes[1, 1].bar(x + width/2, in_values, width, label='In-sample', color='blue', alpha=0.7)

axes[1, 1].set_title('Métriques d\'erreur comparatives')
axes[1, 1].set_ylabel('Valeur de l\'erreur')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(metrics)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Ajout des valeurs sur les barres
for bar in bars1:
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{height:.3f}', ha='center', va='bottom')
for bar in bars2:
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{height:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("📋 COMPARAISON OUT-OF-SAMPLE vs IN-SAMPLE")
print("="*70)
print("\n📊 Résultats simulés:")
print(f"  Out-of-sample MAE: {out_mae:.3f}")
print(f"  In-sample MAE: {in_mae:.3f}")
print(f"  Différence: {((out_mae - in_mae) / in_mae * 100):+.1f}%")
print("\n✅ Points clés:")
print("  1. Out-of-sample: évaluation réaliste de la capacité prédictive")
print("  2. In-sample: évaluation optimiste, utile pour la calibration")
print("  3. L'écart reflète le challenge réel de la prédiction")
print("  4. In-sample inclut information future → erreurs plus faibles")

## Démonstration des classes de validation croisée pour données de panel

Les données de panel combinent plusieurs entités observées dans le temps. Les classes `PanelOutOfSampleSplit` et `PanelInSampleSplit` gèrent cette complexité en appliquant la logique de validation à chaque entité tout en permettant l'agrégation des résultats.

### 4.1 PanelOutOfSampleSplit - Validation hors échantillon pour données de panel

Cette classe applique la logique out-of-sample à chaque entité du panel, respectant l'ordre temporel au sein de chaque entité.

In [ ]:
def analyze_panel_split(X, train_indices, test_indices, split_name, max_entities_display=5):
    """Fonction pour analyser les splits de panel."""
    # Extraction des entités présentes dans train et test
    train_entities = X.iloc[train_indices].index.get_level_values('entity').unique()
    test_entities = X.iloc[test_indices].index.get_level_values('entity').unique()
    
    print(f"\n📊 {split_name}:")
    print(f"  - Observations d'entraînement: {len(train_indices)}")
    print(f"  - Observations de test: {len(test_indices)}")
    print(f"  - Entités dans train: {len(train_entities)} {list(train_entities[:max_entities_display])}")
    print(f"  - Entités dans test: {len(test_entities)} {list(test_entities[:max_entities_display])}")
    
    # Analyse par entité
    if len(test_entities) <= max_entities_display:
        for entity in test_entities:
            entity_train = X.iloc[train_indices].xs(entity, level='entity', drop_level=False)
            entity_test = X.iloc[test_indices].xs(entity, level='entity', drop_level=False)
            
            if len(entity_train) > 0 and len(entity_test) > 0:
                train_dates = entity_train.index.get_level_values('date')
                test_dates = entity_test.index.get_level_values('date')
                gap_days = (test_dates[0] - train_dates[-1]).days
                print(f"    {entity}: Train {train_dates[0].date()}→{train_dates[-1].date()}, Test {test_dates[0].date()}→{test_dates[-1].date()}, Gap {gap_days}j")

print("🔍 DÉMONSTRATION: PanelOutOfSampleSplit")
print("="*70)

# Utilisation du panel équilibré pour les démonstrations
X_panel = panel_balanced[['value']]
print(f"Panel utilisé: {X_panel.shape[0]} observations, {len(X_panel.index.get_level_values('entity').unique())} entités")
print(f"Période: {X_panel.index.get_level_values('date').min().date()} à {X_panel.index.get_level_values('date').max().date()}")

# Configuration 1: Split basique avec n_splits
print(f"\n{'='*50}")
print("CONFIGURATION 1: Panel out-of-sample basique")
print(f"{'='*50}")

panel_splitter1 = PanelOutOfSampleSplit(n_splits=3, test_size=10)
print("📊 Validation croisée avec 3 splits, 10 observations de test par entité")

for i, (train_idx, test_idx) in enumerate(panel_splitter1.split(X_panel)):
    analyze_panel_split(X_panel, train_idx, test_idx, f"Panel split {i+1}")
    if i >= 2:  # Limiter l'affichage
        break

# Configuration 2: Avec gap
print(f"\n{'='*50}")
print("CONFIGURATION 2: Panel avec gap")
print(f"{'='*50}")

panel_splitter2 = PanelOutOfSampleSplit(n_splits=2, test_size=8, gap=5)
print("⚠️  Gap de 5 jours entre train et test pour chaque entité")

for i, (train_idx, test_idx) in enumerate(panel_splitter2.split(X_panel)):
    analyze_panel_split(X_panel, train_idx, test_idx, f"Panel avec gap {i+1}")

# Configuration 3: Test sur dates spécifiques
print(f"\n{'='*50}")
print("CONFIGURATION 3: Test sur dates spécifiques (panel)")
print(f"{'='*50}")

specific_panel_dates = ['2020-03-01', '2020-04-15']
panel_splitter3 = PanelOutOfSampleSplit(test_indices=specific_panel_dates, test_size=7, gap=2)
print(f"🎯 Tests sur: {specific_panel_dates} pour toutes les entités")

for i, (train_idx, test_idx) in enumerate(panel_splitter3.split(X_panel)):
    analyze_panel_split(X_panel, train_idx, test_idx, f"Dates spécifiques {i+1}")

# Configuration 4: Fenêtre d'entraînement limitée
print(f"\n{'='*50}")
print("CONFIGURATION 4: Fenêtre d'entraînement limitée (panel)")
print(f"{'='*50}")

panel_splitter4 = PanelOutOfSampleSplit(n_splits=2, test_size=6, max_train_size=30, gap=1)
print("📏 max_train_size = 30 observations par entité")

for i, (train_idx, test_idx) in enumerate(panel_splitter4.split(X_panel)):
    analyze_panel_split(X_panel, train_idx, test_idx, f"Rolling window panel {i+1}")

In [ ]:
# Visualisation des splits de panel
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Visualisation des splits PanelOutOfSampleSplit', fontsize=16)

# Configuration pour la visualisation
entities_to_plot = entities_small[:3]  # Première 3 entités pour la clarté
colors_entities = ['blue', 'red', 'green']

# Configuration 1: Split basique
ax = axes[0, 0]
for i, (train_idx, test_idx) in enumerate(panel_splitter1.split(X_panel)):
    if i > 0:  # Seulement le premier split pour la clarté
        break
    
    for j, entity in enumerate(entities_to_plot):
        try:
            # Données complètes de l'entité
            entity_data = X_panel.xs(entity, level='entity')
            ax.plot(entity_data.index, entity_data['value'], 
                   color=colors_entities[j], alpha=0.3, linewidth=1, 
                   label=f'{entity} (complet)' if i == 0 else "")
            
            # Données d'entraînement
            entity_train_data = X_panel.iloc[train_idx].xs(entity, level='entity', drop_level=False)
            if len(entity_train_data) > 0:
                train_dates = entity_train_data.index.get_level_values('date')
                ax.plot(train_dates, entity_train_data['value'], 
                       color=colors_entities[j], alpha=0.8, linewidth=2,
                       label=f'{entity} Train' if i == 0 else "")
            
            # Données de test
            entity_test_data = X_panel.iloc[test_idx].xs(entity, level='entity', drop_level=False)
            if len(entity_test_data) > 0:
                test_dates = entity_test_data.index.get_level_values('date')
                ax.scatter(test_dates, entity_test_data['value'], 
                          color=colors_entities[j], s=40, alpha=0.9,
                          edgecolors='black', linewidth=0.5,
                          label=f'{entity} Test' if i == 0 else "")
        except:
            pass

ax.set_title('Panel split basique')
ax.set_ylabel('Valeur')
ax.grid(True, alpha=0.3)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

# Configuration 2: Avec gap
ax = axes[0, 1]
for i, (train_idx, test_idx) in enumerate(panel_splitter2.split(X_panel)):
    if i > 0:  # Seulement le premier split
        break
    
    for j, entity in enumerate(entities_to_plot):
        try:
            entity_data = X_panel.xs(entity, level='entity')
            ax.plot(entity_data.index, entity_data['value'], 
                   color=colors_entities[j], alpha=0.3, linewidth=1)
            
            entity_train_data = X_panel.iloc[train_idx].xs(entity, level='entity', drop_level=False)
            if len(entity_train_data) > 0:
                train_dates = entity_train_data.index.get_level_values('date')
                ax.plot(train_dates, entity_train_data['value'], 
                       color=colors_entities[j], alpha=0.8, linewidth=2)
            
            entity_test_data = X_panel.iloc[test_idx].xs(entity, level='entity', drop_level=False)
            if len(entity_test_data) > 0:
                test_dates = entity_test_data.index.get_level_values('date')
                ax.scatter(test_dates, entity_test_data['value'], 
                          color=colors_entities[j], s=40, alpha=0.9,
                          edgecolors='black', linewidth=0.5)
                
                # Visualisation du gap
                if len(entity_train_data) > 0:
                    gap_start = train_dates[-1]
                    gap_end = test_dates[0]
                    ax.axvspan(gap_start, gap_end, alpha=0.2, color='yellow')
        except:
            pass

ax.set_title('Panel avec gap=5')
ax.set_ylabel('Valeur')
ax.grid(True, alpha=0.3)

# Configuration 3: Dates spécifiques
ax = axes[1, 0]
for i, (train_idx, test_idx) in enumerate(panel_splitter3.split(X_panel)):
    if i > 1:  # Limite à 2 splits
        break
    
    for j, entity in enumerate(entities_to_plot):
        try:
            entity_data = X_panel.xs(entity, level='entity')
            ax.plot(entity_data.index, entity_data['value'], 
                   color=colors_entities[j], alpha=0.3, linewidth=1)
            
            entity_train_data = X_panel.iloc[train_idx].xs(entity, level='entity', drop_level=False)
            if len(entity_train_data) > 0:
                train_dates = entity_train_data.index.get_level_values('date')
                ax.plot(train_dates, entity_train_data['value'], 
                       color=colors_entities[j], alpha=0.8, linewidth=2)
            
            entity_test_data = X_panel.iloc[test_idx].xs(entity, level='entity', drop_level=False)
            if len(entity_test_data) > 0:
                test_dates = entity_test_data.index.get_level_values('date')
                ax.scatter(test_dates, entity_test_data['value'], 
                          color=colors_entities[j], s=40, alpha=0.9,
                          edgecolors='black', linewidth=0.5,
                          marker='s' if i == 0 else 'o')
        except:
            pass

ax.set_title('Dates spécifiques')
ax.set_ylabel('Valeur')
ax.set_xlabel('Date')
ax.grid(True, alpha=0.3)

# Configuration 4: Fenêtre limitée
ax = axes[1, 1]
for i, (train_idx, test_idx) in enumerate(panel_splitter4.split(X_panel)):
    if i > 0:  # Seulement le premier split
        break
    
    for j, entity in enumerate(entities_to_plot):
        try:
            entity_data = X_panel.xs(entity, level='entity')
            ax.plot(entity_data.index, entity_data['value'], 
                   color=colors_entities[j], alpha=0.3, linewidth=1)
            
            entity_train_data = X_panel.iloc[train_idx].xs(entity, level='entity', drop_level=False)
            if len(entity_train_data) > 0:
                train_dates = entity_train_data.index.get_level_values('date')
                ax.plot(train_dates, entity_train_data['value'], 
                       color=colors_entities[j], alpha=0.8, linewidth=2)
            
            entity_test_data = X_panel.iloc[test_idx].xs(entity, level='entity', drop_level=False)
            if len(entity_test_data) > 0:
                test_dates = entity_test_data.index.get_level_values('date')
                ax.scatter(test_dates, entity_test_data['value'], 
                          color=colors_entities[j], s=40, alpha=0.9,
                          edgecolors='black', linewidth=0.5)
        except:
            pass

ax.set_title('Fenêtre limitée (max_train_size=30)')
ax.set_ylabel('Valeur')
ax.set_xlabel('Date')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 4.2 Classes spécialisées pour le traitement par entité

Les classes `PanelOutOfSampleSplitPerEntity` et `PanelInSampleSplitPerEntity` permettent de traiter chaque entité séparément, ce qui est utile pour l'analyse individuelle et le traitement parallèle.

In [ ]:
print("🔍 DÉMONSTRATION: Classes spécialisées par entité")
print("="*70)

# Comparaison des approches PanelOutOfSampleSplit vs PanelOutOfSampleSplitPerEntity
print("\n📊 Comparaison: Agrégé vs Par entité")
print("-" * 50)

# Splitter agrégé (standard)
panel_agg_splitter = PanelOutOfSampleSplit(n_splits=2, test_size=5, gap=2)

# Splitter par entité
panel_per_entity_splitter = PanelOutOfSampleSplitPerEntity(n_splits=2, test_size=5, gap=2)

print("\n1️⃣  APPROCHE AGRÉGÉE (PanelOutOfSampleSplit):")
split_count = 0
for train_idx, test_idx in panel_agg_splitter.split(X_panel):
    split_count += 1
    entities_in_test = X_panel.iloc[test_idx].index.get_level_values('entity').unique()
    print(f"  Split {split_count}: {len(test_idx)} observations de test, {len(entities_in_test)} entités")
    if split_count >= 2:
        break

print(f"\n  📈 Total: {split_count} splits avec toutes les entités mélangées")

print("\n2️⃣  APPROCHE PAR ENTITÉ (PanelOutOfSampleSplitPerEntity):")
split_count = 0
entity_splits = {}

for train_idx, test_idx in panel_per_entity_splitter.split(X_panel):
    split_count += 1
    # Identifier l'entité de ce split
    test_entity = X_panel.iloc[test_idx].index.get_level_values('entity').unique()[0]
    train_entity = X_panel.iloc[train_idx].index.get_level_values('entity').unique()[0] if len(train_idx) > 0 else "N/A"
    
    if test_entity not in entity_splits:
        entity_splits[test_entity] = 0
    entity_splits[test_entity] += 1
    
    print(f"  Split {split_count}: Entité {test_entity}, {len(train_idx)} train, {len(test_idx)} test")
    
    if split_count >= 8:  # Limiter l'affichage
        print("  ... (splits supplémentaires)")
        break

print(f"\n  📈 Total: {split_count}+ splits individuels par entité")
print(f"  📊 Répartition par entité: {dict(entity_splits)}")

# Démonstration avec In-sample per entity
print(f"\n{'='*50}")
print("DÉMONSTRATION: PanelInSampleSplitPerEntity")
print(f"{'='*50}")

panel_insample_per_entity = PanelInSampleSplitPerEntity(test_indices=['2020-03-01'], test_size=7)

print("🎯 Test sur 2020-03-01 avec validation in-sample par entité:")
entity_results = {}

for train_idx, test_idx in panel_insample_per_entity.split(X_panel):
    test_entity = X_panel.iloc[test_idx].index.get_level_values('entity').unique()[0]
    train_data = X_panel.iloc[train_idx]
    test_data = X_panel.iloc[test_idx]
    
    # Vérification que le test est inclus dans l'entraînement
    test_in_train = all(idx in train_idx for idx in test_idx)
    
    entity_results[test_entity] = {
        'train_size': len(train_idx),
        'test_size': len(test_idx),
        'test_in_train': test_in_train,
        'train_period': (train_data.index.get_level_values('date').min().date(), 
                        train_data.index.get_level_values('date').max().date()),
        'test_period': (test_data.index.get_level_values('date').min().date(),
                       test_data.index.get_level_values('date').max().date())
    }
    
    print(f"  {test_entity}: Train {entity_results[test_entity]['train_size']} obs, "
          f"Test {entity_results[test_entity]['test_size']} obs, "
          f"Test in Train: {'✅' if test_in_train else '❌'}")

# Statistiques de comparaison
print(f"\n{'='*50}")
print("📊 STATISTIQUES COMPARATIVES")
print(f"{'='*50}")

def calculate_split_statistics(splitter, X, split_type_name):
    """Calcule des statistiques sur les splits."""
    total_splits = 0
    total_train_obs = 0
    total_test_obs = 0
    entities_seen = set()
    
    for train_idx, test_idx in splitter.split(X):
        total_splits += 1
        total_train_obs += len(train_idx)
        total_test_obs += len(test_idx)
        
        test_entities = X.iloc[test_idx].index.get_level_values('entity').unique()
        entities_seen.update(test_entities)
        
        if total_splits >= 10:  # Limite pour éviter trop de calculs
            break
    
    return {
        'type': split_type_name,
        'total_splits': total_splits,
        'avg_train_size': total_train_obs / total_splits if total_splits > 0 else 0,
        'avg_test_size': total_test_obs / total_splits if total_splits > 0 else 0,
        'unique_entities': len(entities_seen)
    }

# Calcul des statistiques
stats_agg = calculate_split_statistics(panel_agg_splitter, X_panel, "Agrégé")
stats_per_entity = calculate_split_statistics(panel_per_entity_splitter, X_panel, "Par entité")

print(f"\n📈 Résultats (sur {min(10, stats_agg['total_splits'])} premiers splits):")
print(f"  Approche agrégée:")
print(f"    - Splits: {stats_agg['total_splits']}")
print(f"    - Taille moyenne train: {stats_agg['avg_train_size']:.1f}")
print(f"    - Taille moyenne test: {stats_agg['avg_test_size']:.1f}")
print(f"    - Entités uniques vues: {stats_agg['unique_entities']}")

print(f"  Approche par entité:")
print(f"    - Splits: {stats_per_entity['total_splits']}")
print(f"    - Taille moyenne train: {stats_per_entity['avg_train_size']:.1f}")
print(f"    - Taille moyenne test: {stats_per_entity['avg_test_size']:.1f}")
print(f"    - Entités uniques vues: {stats_per_entity['unique_entities']}")

print(f"\n✅ Avantages par approche:")
print(f"  📊 Agrégée: Moins de splits, évaluation globale, plus rapide")
print(f"  🎯 Par entité: Analyse détaillée, traitement parallèle possible, contrôle granulaire")

## Guide pratique et meilleures pratiques

Cette section présente un guide pratique pour choisir la méthode de validation croisée appropriée selon différents scénarios et objectifs d'analyse.

In [ ]:
print("📚 GUIDE PRATIQUE: Choisir la bonne méthode de validation croisée")
print("="*80)

# Matrice de décision
decision_matrix = {
    "Type de données": {
        "Série temporelle unique": ["TSOutOfSampleSplit", "TSInSampleSplit"],
        "Panel data (multi-entités)": ["PanelOutOfSampleSplit", "PanelInSampleSplit", 
                                      "PanelOutOfSampleSplitPerEntity", "PanelInSampleSplitPerEntity"]
    },
    "Objectif d'évaluation": {
        "Prédiction future (production)": ["OutOfSampleSplit"],
        "Analyse historique/calibration": ["InSampleSplit"],
        "Évaluation comparative": ["OutOfSampleSplit", "InSampleSplit"]
    },
    "Contraintes temporelles": {
        "Éviter data leakage": ["gap > 0"],
        "Fenêtre glissante": ["max_train_size < inf"],
        "Dates spécifiques": ["test_indices=[dates]"]
    },
    "Analyse par entité": {
        "Évaluation globale": ["PanelOutOfSampleSplit", "PanelInSampleSplit"],
        "Analyse individuelle": ["PanelOutOfSampleSplitPerEntity", "PanelInSampleSplitPerEntity"],
        "Traitement parallèle": ["PerEntity classes"]
    }
}\n\n# Affichage du guide de décision\nfor category, options in decision_matrix.items():\n    print(f\"\\n🎯 {category.upper()}:\")\n    for scenario, methods in options.items():\n        print(f\"  • {scenario}: {', '.join(methods)}\")\n\n# Exemples de configurations recommandées\nprint(f\"\\n{'='*50}\")\nprint(\"CONFIGURATIONS RECOMMANDÉES\")\nprint(f\"{'='*50}\")\n\nrecommended_configs = [\n    {\n        \"scenario\": \"Prédiction de séries temporelles financières\",\n        \"config\": \"TSOutOfSampleSplit(n_splits=5, test_size=22, gap=1)\",\n        \"rationale\": \"Gap d'1 jour pour éviter le look-ahead bias, test sur ~1 mois\"\n    },\n    {\n        \"scenario\": \"Évaluation de modèles sur panel d'entreprises\",\n        \"config\": \"PanelOutOfSampleSplit(test_size=30, gap=5, max_train_size=252)\",\n        \"rationale\": \"Fenêtre d'1 an, gap de 5 jours, test sur 1 mois\"\n    },\n    {\n        \"scenario\": \"Backtesting historique avec calibration\",\n        \"config\": \"TSInSampleSplit(test_indices=['2020-03-01'], test_size=14)\",\n        \"rationale\": \"Test sur période de crise spécifique, entraînement inclut le futur\"\n    },\n    {\n        \"scenario\": \"Analyse de robustesse par entité\",\n        \"config\": \"PanelOutOfSampleSplitPerEntity(n_splits=3, test_size=10)\",\n        \"rationale\": \"Évaluation individuelle de chaque entité avec 3 périodes de test\"\n    },\n    {\n        \"scenario\": \"Validation avec données haute fréquence\",\n        \"config\": \"TSOutOfSampleSplit(test_size=50, gap=10, max_train_size=1000)\",\n        \"rationale\": \"Gap plus important, fenêtre limitée pour données intraday\"\n    }\n]\n\nfor i, config in enumerate(recommended_configs, 1):\n    print(f\"\\n{i}️⃣  {config['scenario']}:\")\n    print(f\"   📝 Configuration: {config['config']}\")\n    print(f\"   💡 Rationale: {config['rationale']}\")\n\n# Métriques de performance simulées\nprint(f\"\\n{'='*50}\")\nprint(\"EXEMPLE DE PIPELINE D'ÉVALUATION\")\nprint(f\"{'='*50}\")\n\ndef simulate_model_evaluation(X, splitter, model_name=\"Modèle simple\"):\n    \"\"\"Simule l'évaluation d'un modèle avec cross-validation.\"\"\"\n    mae_scores = []\n    mse_scores = []\n    \n    split_count = 0\n    for train_idx, test_idx in splitter.split(X):\n        # Simulation d'entraînement et prédiction\n        X_train = X.iloc[train_idx]\n        X_test = X.iloc[test_idx]\n        \n        # Prédiction naive (moyenne mobile) pour simulation\n        if len(X_train) > 0:\n            if hasattr(X_train.index, 'get_level_values'):\n                # Panel data - moyenne par groupe\n                pred = X_train.groupby(level='entity')['value'].mean().mean()\n            else:\n                # Série temporelle - moyenne des dernières valeurs\n                pred = X_train['value'].tail(min(10, len(X_train))).mean()\n        else:\n            pred = 0\n        \n        # Calcul des erreurs simulées\n        true_values = X_test['value'].values\n        predictions = np.full_like(true_values, pred)\n        \n        # Ajout de bruit réaliste selon le type de validation\n        if 'OutOfSample' in splitter.__class__.__name__:\n            noise_std = 0.8  # Plus d'incertitude out-of-sample\n        else:\n            noise_std = 0.4  # Moins d'incertitude in-sample\n        \n        predictions += np.random.normal(0, noise_std, len(predictions))\n        \n        mae = np.mean(np.abs(true_values - predictions))\n        mse = np.mean((true_values - predictions)**2)\n        \n        mae_scores.append(mae)\n        mse_scores.append(mse)\n        \n        split_count += 1\n        if split_count >= 5:  # Limite pour la démonstration\n            break\n    \n    return {\n        'mae_mean': np.mean(mae_scores),\n        'mae_std': np.std(mae_scores),\n        'mse_mean': np.mean(mse_scores),\n        'mse_std': np.std(mse_scores),\n        'n_splits': len(mae_scores)\n    }\n\n# Comparaison de performance entre différentes méthodes\nprint(\"\\n🔬 Simulation d'évaluation comparative:\")\n\nsplitters_to_compare = [\n    (TSOutOfSampleSplit(n_splits=3, test_size=15, gap=2), \"TS Out-of-Sample\"),\n    (TSInSampleSplit(test_size=15), \"TS In-Sample\"),\n    (PanelOutOfSampleSplit(n_splits=2, test_size=8), \"Panel Out-of-Sample\"),\n    (PanelInSampleSplit(test_size=8), \"Panel In-Sample\")\n]\n\nresults = []\nfor splitter, name in splitters_to_compare:\n    if 'Panel' in name:\n        data = X_panel\n    else:\n        data = X\n    \n    try:\n        result = simulate_model_evaluation(data, splitter, name)\n        result['method'] = name\n        results.append(result)\n        \n        print(f\"\\n  📊 {name}:\")\n        print(f\"     MAE: {result['mae_mean']:.3f} ± {result['mae_std']:.3f}\")\n        print(f\"     MSE: {result['mse_mean']:.3f} ± {result['mse_std']:.3f}\")\n        print(f\"     Splits: {result['n_splits']}\")\n    except Exception as e:\n        print(f\"\\n  ❌ {name}: Erreur - {str(e)[:50]}...\")\n\nprint(f\"\\n{'='*50}\")\nprint(\"POINTS CLÉS À RETENIR\")\nprint(f\"{'='*50}\")\n\nkey_points = [\n    \"Out-of-sample donne une évaluation plus conservative et réaliste\",\n    \"In-sample est utile pour l'analyse historique et la calibration\",\n    \"Le gap prévient le data leakage dans les données haute fréquence\",\n    \"max_train_size permet une validation en fenêtre glissante\",\n    \"Les classes Panel gèrent automatiquement la structure multi-entités\",\n    \"PerEntity permet l'analyse granulaire et le traitement parallèle\",\n    \"test_indices permet de tester sur des événements spécifiques\"\n]\n\nfor i, point in enumerate(key_points, 1):\n    print(f\"  {i}. {point}\")\n\nprint(f\"\\n✅ Le choix de la méthode dépend de votre objectif:\")\nprint(f\"   🎯 Production: Out-of-sample avec gap approprié\")\nprint(f\"   📊 Recherche: In-sample pour analyse historique\")\nprint(f\"   🔬 Robustesse: Comparaison des deux approches\")"

## Conclusion

Ce notebook a présenté de manière exhaustive les classes de validation croisée du module `tsforecast.crossvals`. Voici les points essentiels à retenir :

### Fonctionnalités principales

1. **Classes pour séries temporelles** : `TSOutOfSampleSplit` et `TSInSampleSplit`
   - Respectent l'ordre chronologique
   - Gèrent les gaps pour éviter le data leakage
   - Supportent les fenêtres glissantes et les dates spécifiques

2. **Classes pour données de panel** : `PanelOutOfSampleSplit`, `PanelInSampleSplit` et leurs variantes `PerEntity`
   - Traitent les données multi-entités automatiquement
   - Appliquent la logique temporelle au sein de chaque entité
   - Permettent l'analyse granulaire par entité

3. **Flexibilité de configuration** : 
   - Paramètres `n_splits`, `test_size`, `gap`, `max_train_size`
   - Support des `test_indices` pour des périodes spécifiques
   - Compatible avec l'API sklearn

### Recommandations d'usage

- **Out-of-sample** : Pour l'évaluation réaliste de modèles de production
- **In-sample** : Pour l'analyse historique et la calibration
- **Gap** : Essentiel pour les données haute fréquence
- **PerEntity** : Pour l'analyse détaillée et le traitement parallèle

Ces classes offrent une base solide pour l'évaluation rigoureuse de modèles sur données temporelles et de panel, en respectant les contraintes inhérentes à ce type de données.